<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/YOLO_v5_Wheet_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt